In [1]:
import random
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd
length = 1000
cols = ["Q", "X", "Y", "Z"]
mu = 0
sigma = 5


import pingouin
lst_dct = {col:[] for col in cols }
for i in range(length):
    lst_dct["Q"].append(50 + np.random.normal(mu, sigma))
    lst_dct["X"].append(5 * lst_dct["Q"][-1] + 10 + np.random.normal(mu, sigma ))
    lst_dct["Y"].append(lst_dct["Q"][-1] * -3 + 20 +  np.random.normal(mu, sigma))
    lst_dct["Z"].append(5 * lst_dct["X"][-1] + 10 * lst_dct["Y"][-1] +  np.random.normal(mu, 3 * sigma))

df = pd.DataFrame(lst_dct)
keys = ["X",
        "Y",
        "Z",
        "Q"]
dag_keys = keys
df

,Q,X,Y,Z
0,48.805608,249.416014,-130.668481,-66.840088
1,54.609501,284.443512,-141.142266,22.466170
2,51.073867,270.069942,-135.073359,-13.840069
3,47.180271,246.909622,-125.426971,-30.450646
4,53.290175,274.246077,-150.622983,-137.086769
...,...,...,...,...
995,55.884150,291.772324,-155.293021,-90.498218
996,51.588314,264.745830,-128.438726,49.719846
997,37.453228,201.477779,-98.491964,24.632904
998,46.987903,249.074239,-117.590933,43.660942


In [2]:
import pingouin

undirected_graph = {key:[] for key in df.keys()}
for x in undirected_graph:
    remaining_vars = [y for y in df.keys() if y != x]
    for y in remaining_vars:
        undirected_graph[x].append(y)

undirected_graph           

{'Q': ['X', 'Y', 'Z'],
 'X': ['Q', 'Y', 'Z'],
 'Y': ['Q', 'X', 'Z'],
 'Z': ['Q', 'X', 'Y']}

In [4]:
p_val = .001
    
for x in undirected_graph.keys():
    ys = undirected_graph[x]
    for y in ys:
        if x != y:
            # first check for correlation with no controls
            test = df.partial_corr(x = x, y = y, covar = [])
            if test["p-val"].values[0] > p_val:
                undirected_graph[x].remove(y)
            # if correlated check for deseparation controlling for other variables
            else:
                ############## make recursive function #############
                
                control_vars = [z for z in df.keys() if z != y and z != x]
                for z in control_vars:
                    test = df.partial_corr(x = x, y = y, covar = z)
                    if test["p-val"].values[0] > p_val:
                        undirected_graph[x].remove(y)
                ######################## end #######################                        

                # next step will require recursion like when we
                # mapped the partial correlations for all 
                # combinations of control variables
                
undirected_graph

{'Q': ['X', 'Y', 'Z'], 'X': ['Q', 'Z'], 'Y': ['Q', 'Z'], 'Z': ['Q', 'X', 'Y']}